In [1]:
import math
import numpy as np
import pandas as pd

# **1. Imports and load NMDB data**

In [2]:
# Load main NMDB dataset
df = pd.read_csv("Results/DataStudy.csv")
# Parse datetime
df["DATETIME"] = pd.to_datetime(df["DATETIME"])
# Ensure timezone-naive index (if it was timezone-aware, drop tz; if not, this is harmless)
try:
    df["DATETIME"] = df["DATETIME"].dt.tz_convert(None)
except TypeError:
    # Already naive, nothing to do
    pass
df.set_index("DATETIME", inplace=True)
df = df.sort_index()
df.head()

,MXCO,JUNG1,LMKS,NEWK,KERG,OULU,DOMC,INVK,APTY,AATB
DATETIME,,,,,,,,,,
2018-01-01 00:00:00,231.000,375.029,474.079,NaN,234.337,111.260,20.289,196.15,179.579,1416.45
2018-01-01 00:02:00,233.000,375.602,475.306,NaN,233.188,107.716,21.116,196.88,178.488,1424.00
2018-01-01 00:04:00,232.250,372.678,474.119,NaN,233.306,110.502,20.536,196.99,178.687,1436.40
2018-01-01 00:06:00,228.417,364.417,472.085,NaN,234.694,109.483,20.876,197.58,179.851,1415.80
2018-01-01 00:08:00,227.500,365.883,474.529,NaN,234.920,110.824,20.992,199.17,175.817,1427.35


# **2. Station list and basic time-step check**

In [3]:
# Stations to use in the Sierra method
STATIONS = ["MXCO", "JUNG1", "LMKS", "KERG", "OULU", "NEWK", "DOMC", "INVK", "APTY", "AATB"]
# Quick check: ensure all stations exist
missing = [s for s in STATIONS if s not in df.columns]
if missing:
    raise ValueError(f"Missing station columns in df: {missing}")
# Check approximate time step (in minutes)
dt_minutes = (df.index[1] - df.index[0]).total_seconds() / 60.0
print(f"Approx. time step: {dt_minutes:.2f} minutes")

Approx. time step: 2.00 minutes


# **3. Core helper functions (deltaN + complexity + robust z)**

## **3.1 Background and deltaN**

In [4]:
def make_deltaN(df, station, bg_window_hours=72):
    """
    Build a panel for one station with:
      - counts
      - counts_clean (spikes removed)
      - bg (robust background via rolling median)
      - deltaN (% deviation from bg)
    """
    series = df[station].astype(float)

    # Remove very large spikes (crude but safe)
    med = series.median()
    mad = (series - med).abs().median()
    clean = series.mask((series - med).abs() > 6 * mad)

    # Time step and background window
    dt_minutes = (df.index[1] - df.index[0]).total_seconds() / 60.0
    pts_per_hour = int(round(60.0 / dt_minutes))
    bg_window = int(bg_window_hours * pts_per_hour)

    bg = clean.rolling(window=bg_window,
                       center=True,
                       min_periods=int(0.5 * bg_window)).median()

    deltaN = 100.0 * (clean - bg) / bg

    out = pd.DataFrame({
        "counts": series,
        "counts_clean": clean,
        "bg": bg,
        "deltaN": deltaN
    })

    return out

## **3.2 Complexity measures: permutation entropy and Katz FD**

In [5]:
def permutation_entropy(x, m=3, delay=1):
    """
    Normalized permutation entropy in [0, 1].
    x: 1D array-like
    """
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    n = len(x)
    if n < m * delay:
        return np.nan

    patterns = []
    for i in range(n - (m - 1) * delay):
        window = x[i:i + m * delay:delay]
        ranks = np.argsort(window)
        patterns.append(tuple(ranks))

    if len(patterns) == 0:
        return np.nan

    patterns = np.array(patterns, dtype=object)
    _, counts = np.unique(patterns, return_counts=True)
    probs = counts / counts.sum()

    pe = -np.sum(probs * np.log(probs))
    pe_norm = pe / math.log(math.factorial(m))

    return pe_norm

In [6]:
def katz_fd(x):
    """
    Katz fractal dimension for a 1D series.
    """
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    n = len(x)
    if n < 2:
        return np.nan

    diffs = np.diff(x)
    L = np.sum(np.sqrt(1.0 + diffs**2))

    t = np.arange(n, dtype=float)
    dists = np.sqrt((t - t[0])**2 + (x - x[0])**2)
    d = np.max(dists)

    if d == 0 or L == 0:
        return np.nan

    return math.log10(n) / (math.log10(n) + math.log10(d / L))

## **3.3 Rolling complexity panel**

In [7]:
def compute_complexity_panel(st_df, window_hours=3, m=3, delay=1):
    """
    Add PE and KFD in a sliding window over 'counts_clean'.
    """
    dt_minutes = (st_df.index[1] - st_df.index[0]).total_seconds() / 60.0
    pts_per_hour = int(round(60.0 / dt_minutes))
    win = int(window_hours * pts_per_hour)

    roll = st_df["counts_clean"].rolling(window=win,
                                         min_periods=int(0.5 * win))

    pe = roll.apply(lambda x: permutation_entropy(x, m=m, delay=delay),
                    raw=True)
    kfd = roll.apply(lambda x: katz_fd(x),
                     raw=True)

    out = st_df.copy()
    out["PE"] = pe
    out["KFD"] = kfd

    return out

## **3.4 Robust z-score and complexity anomaly**

In [8]:
def robust_z(series, ref_mask):
    """
    Robust z-score (median/MAD) over a reference subset.
    """
    ref = series[ref_mask].dropna()
    med = ref.median()
    mad = (ref - med).abs().median()

    if mad == 0 or np.isnan(mad):
        return pd.Series(np.nan, index=series.index)

    return (series - med) / mad

In [9]:
def add_complexity_scores(st_panel,
                          quiet_start="2019-01-15",
                          quiet_end="2019-03-15"):
    """
    Add z_PE, z_KFD and A_complex to a station panel.
    quiet_start/quiet_end define the reference quiet period.
    """
    panel = st_panel.copy()
    ref_mask = (panel.index >= quiet_start) & (panel.index < quiet_end)

    panel["z_PE"] = robust_z(panel["PE"], ref_mask)
    panel["z_KFD"] = robust_z(panel["KFD"], ref_mask)

    panel["A_complex"] = panel[["z_PE", "z_KFD"]].abs().max(axis=1)

    return panel

# **4. Build per-station panels (deltaN + complexity)**

In [10]:
station_panels = {}

for st in STATIONS:
    print(f"Building panel for {st} ...")
    base = make_deltaN(df, st)
    base = compute_complexity_panel(base, window_hours=3)
    base = add_complexity_scores(base,
                                 quiet_start="2019-01-15",
                                 quiet_end="2019-03-15")
    base["station_name"] = st
    station_panels[st] = base

# Quick check
station_panels["JUNG1"][["counts", "deltaN", "PE", "KFD", "A_complex"]].head()

Building panel for MXCO ...
Building panel for JUNG1 ...
Building panel for LMKS ...
Building panel for KERG ...
Building panel for OULU ...
Building panel for NEWK ...
Building panel for DOMC ...
Building panel for INVK ...
Building panel for APTY ...
Building panel for AATB ...


,counts,deltaN,PE,KFD,A_complex
DATETIME,,,,,
2018-01-01 00:00:00,375.029,1.608955,NaN,NaN,NaN
2018-01-01 00:02:00,375.602,1.764339,NaN,NaN,NaN
2018-01-01 00:04:00,372.678,0.972668,NaN,NaN,NaN
2018-01-01 00:06:00,364.417,-1.265017,NaN,NaN,NaN
2018-01-01 00:08:00,365.883,-0.868357,NaN,NaN,NaN


# **5. Sierra FD detector (single station) and level definitions**

In [11]:
def detect_fd_events(
    station_df,
    a_thresh=3.0,
    min_a_duration_hours=3.0,
    min_drop_percent=2.0,
    max_time_to_min_hours=48.0,
    recovery_fraction=0.5,
    max_recovery_days=7.0,
    bg_lookback_hours=24.0
):
    """
    Sierra FD detector for a single station:
      - station_df must have columns: deltaN, A_complex
      - and a regular DateTimeIndex (naive or tz-aware).
    """
    df = station_df.copy()

    if "deltaN" not in df.columns or "A_complex" not in df.columns:
        raise ValueError("station_df must have 'deltaN' and 'A_complex'")

    dt_minutes = (df.index[1] - df.index[0]).total_seconds() / 60.0
    pts_per_hour = max(int(round(60.0 / dt_minutes)), 1)

    min_a_pts        = int(round(min_a_duration_hours * pts_per_hour))
    max_to_min_pts   = int(round(max_time_to_min_hours * pts_per_hour))
    max_recovery_pts = int(round(max_recovery_days * 24.0 * pts_per_hour))
    bg_lookback_pts  = int(round(bg_lookback_hours * pts_per_hour))

    high_a = df["A_complex"] > a_thresh
    events = []

    if not high_a.any():
        return pd.DataFrame(columns=[
            "station", "onset_time", "min_time", "rec_time",
            "drop_percent", "A_complex_max", "bg_level_pre"
        ])

    idx = np.where(high_a.values)[0]

    # group contiguous indices into segments
    segments = []
    start = idx[0]
    prev = idx[0]
    for i in idx[1:]:
        if i == prev + 1:
            prev = i
        else:
            segments.append((start, prev))
            start = i
            prev = i
    segments.append((start, prev))

    # station name
    if "station_name" in df.columns:
        station_name = str(df["station_name"].iloc[0])
    else:
        station_name = "unknown"

    for (s0, s1) in segments:
        length = s1 - s0 + 1
        if length < min_a_pts:
            continue

        onset_idx = s0

        i_start_bg = max(0, onset_idx - bg_lookback_pts)
        i_end_bg   = max(0, onset_idx - 1)
        if i_end_bg <= i_start_bg:
            continue

        bg_deltaN = df["deltaN"].iloc[i_start_bg:i_end_bg]
        if bg_deltaN.dropna().empty:
            continue

        level_pre = bg_deltaN.median()

        i_min_search_end = min(len(df) - 1, onset_idx + max_to_min_pts)
        seg_deltaN = df["deltaN"].iloc[onset_idx:i_min_search_end + 1]
        if seg_deltaN.dropna().empty:
            continue

        i_local_min = seg_deltaN.idxmin()
        j_min = df.index.get_loc(i_local_min)
        min_val = df.loc[i_local_min, "deltaN"]
        drop = level_pre - min_val

        if drop < min_drop_percent:
            continue

        i_rec_start = j_min
        i_rec_end = min(len(df) - 1, j_min + max_recovery_pts)
        rec_seg = df["deltaN"].iloc[i_rec_start:i_rec_end + 1]
        if rec_seg.dropna().empty:
            continue

        target_level = level_pre - (1.0 - recovery_fraction) * drop
        rec_mask = rec_seg >= target_level
        if rec_mask.any():
            rec_time = rec_seg.index[rec_mask.argmax()]
        else:
            rec_time = np.nan

        a_max = df["A_complex"].iloc[s0:s1 + 1].max()

        events.append({
            "station": station_name,
            "onset_time": df.index[onset_idx],
            "min_time": i_local_min,
            "rec_time": rec_time,
            "drop_percent": float(drop),
            "A_complex_max": float(a_max),
            "bg_level_pre": float(level_pre),
        })

    return pd.DataFrame(events)

In [12]:
SIERRA_LEVELS = {
    "strict": {"a_thresh": 3.0, "min_drop_percent": 2.0},
    "medium": {"a_thresh": 2.0, "min_drop_percent": 1.5},
    "loose":  {"a_thresh": 1.5, "min_drop_percent": 1.0},
}

# **6. Run Sierra detector for all stations and levels**

In [13]:
events_by_level = {}   # level_name -> DataFrame with all stations

for level_name, params in SIERRA_LEVELS.items():
    print(f"\nRunning Sierra level: {level_name}")
    all_events = []
    for st, panel in station_panels.items():
        ev = detect_fd_events(
            panel,
            a_thresh=params["a_thresh"],
            min_drop_percent=params["min_drop_percent"]
        )
        ev["level"] = level_name
        all_events.append(ev)

    if all_events:
        ev_all = pd.concat(all_events, ignore_index=True)
    else:
        ev_all = pd.DataFrame(columns=[
            "station", "onset_time", "min_time", "rec_time",
            "drop_percent", "A_complex_max", "bg_level_pre", "level"
        ])

    # Ensure naive datetimes
    for col in ["onset_time", "min_time", "rec_time"]:
        if col in ev_all.columns and pd.api.types.is_datetime64tz_dtype(ev_all[col]):
            ev_all[col] = ev_all[col].dt.tz_convert(None)

    events_by_level[level_name] = ev_all

# Quick check
for lvl, ev in events_by_level.items():
    print(f"{lvl}: {len(ev)} events")
    display(ev.head())


Running Sierra level: strict


/var/folders/8n/h_rtmhz56bd84hgb5_cp68fc0000gn/T/ipykernel_15657/2661864651.py:25: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if col in ev_all.columns and pd.api.types.is_datetime64tz_dtype(ev_all[col]):



Running Sierra level: medium


/var/folders/8n/h_rtmhz56bd84hgb5_cp68fc0000gn/T/ipykernel_15657/2661864651.py:25: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if col in ev_all.columns and pd.api.types.is_datetime64tz_dtype(ev_all[col]):



Running Sierra level: loose
strict: 1406 events


...

,station,onset_time,min_time,rec_time,drop_percent,A_complex_max,bg_level_pre,level
0,MXCO,2018-01-13 10:52:00,2018-01-15 07:28:00,2018-01-15 07:30:00,3.017396,5.828022,0.071706,strict
1,MXCO,2018-01-28 10:54:00,2018-01-30 03:04:00,2018-01-30 03:06:00,2.715966,5.470556,-0.000434,strict
2,MXCO,2018-03-20 06:54:00,2018-03-20 16:42:00,2018-03-20 16:44:00,3.957647,5.525199,0.000428,strict
3,MXCO,2018-04-16 18:08:00,2018-04-18 13:30:00,2018-04-18 13:32:00,3.932926,4.393013,-0.000214,strict
4,MXCO,2018-04-25 02:20:00,2018-04-25 03:26:00,2018-04-25 03:28:00,3.288606,5.082601,-0.072858,strict


medium: 8839 events


,station,onset_time,min_time,rec_time,drop_percent,A_complex_max,bg_level_pre,level
0,MXCO,2018-01-06 22:48:00,2018-01-06 22:58:00,2018-01-06 23:00:00,6.148011,5.069751,0.036456,medium
1,MXCO,2018-01-09 03:08:00,2018-01-10 01:48:00,2018-01-10 01:50:00,13.056328,4.242166,-0.144521,medium
2,MXCO,2018-01-13 10:44:00,2018-01-15 07:28:00,2018-01-15 07:30:00,3.017396,5.828022,0.071706,medium
3,MXCO,2018-01-28 09:52:00,2018-01-30 03:04:00,2018-01-30 03:06:00,2.716616,5.470556,0.000217,medium
4,MXCO,2018-01-29 05:28:00,2018-01-31 00:16:00,2018-01-31 00:18:00,3.369039,3.658180,0.035996,medium


loose: 18580 events


,station,onset_time,min_time,rec_time,drop_percent,A_complex_max,bg_level_pre,level
0,MXCO,2018-01-04 17:44:00,2018-01-06 01:44:00,2018-01-06 01:48:00,6.773046,4.000969,-0.036641,loose
1,MXCO,2018-01-06 00:36:00,2018-01-06 01:44:00,2018-01-06 01:48:00,6.736591,4.327187,-0.073096,loose
2,MXCO,2018-01-06 22:28:00,2018-01-06 22:58:00,2018-01-06 23:00:00,6.148011,5.069751,0.036456,loose
3,MXCO,2018-01-07 17:22:00,2018-01-09 04:08:00,2018-01-09 04:10:00,3.326986,4.522868,0.108460,loose
4,MXCO,2018-01-09 03:06:00,2018-01-10 01:48:00,2018-01-10 01:50:00,13.056328,4.242166,-0.144521,loose


# **7. Build coincidence tables (multi-station groups) for each level**

In [14]:
def build_coincidence_table(events_df,
                            time_col="min_time",
                            coincidence_window_hours=12):
    """
    Group events from multiple stations into coincident FD candidates.
    """
    if events_df.empty:
        return pd.DataFrame(columns=[
            "repr_time", "n_stations", "stations",
            "drop_mean", "drop_max",
            "A_complex_mean", "A_complex_max"
        ])

    ev = events_df.sort_values(time_col).reset_index(drop=True)
    dt = pd.to_timedelta(coincidence_window_hours, unit="h")

    groups = []
    current_group = [0]
    current_ref_time = ev.loc[0, time_col]

    for i in range(1, len(ev)):
        t = ev.loc[i, time_col]
        if abs(t - current_ref_time) <= dt:
            current_group.append(i)
            current_ref_time = ev.loc[current_group, time_col].median()
        else:
            groups.append(current_group)
            current_group = [i]
            current_ref_time = t
    groups.append(current_group)

    rows = []
    for g in groups:
        g_ev = ev.loc[g]
        rows.append({
            "repr_time": g_ev[time_col].median(),
            "n_stations": g_ev["station"].nunique(),
            "stations": ",".join(sorted(g_ev["station"].unique())),
            "drop_mean": g_ev["drop_percent"].mean(),
            "drop_max": g_ev["drop_percent"].max(),
            "A_complex_mean": g_ev["A_complex_max"].mean(),
            "A_complex_max": g_ev["A_complex_max"].max(),
        })

    out = pd.DataFrame(rows).sort_values("repr_time").reset_index(drop=True)
    return out

In [15]:
coincidence_by_level = {}

for level_name, ev in events_by_level.items():
    coinc = build_coincidence_table(ev,
                                    time_col="min_time",
                                    coincidence_window_hours=12)
    coincidence_by_level[level_name] = coinc
    print(f"\n=== {level_name.upper()} SIERRA — COINCIDENCE TABLE ({len(coinc)} events) ===")
    display(coinc.head())


=== STRICT SIERRA — COINCIDENCE TABLE (919 events) ===


,repr_time,n_stations,stations,drop_mean,drop_max,A_complex_mean,A_complex_max
0,2018-01-01 15:48:00,1,INVK,4.077303,4.077303,5.525709,5.525709
1,2018-01-02 19:30:00,1,JUNG1,4.764506,4.925805,6.694942,8.011326
2,2018-01-04 16:48:00,1,JUNG1,3.167172,3.167172,6.417919,6.417919
3,2018-01-10 10:09:00,2,"INVK,JUNG1",2.860242,3.329036,5.558226,6.310481
4,2018-01-12 12:04:00,1,APTY,3.209376,3.209376,6.366117,6.366117



=== MEDIUM SIERRA — COINCIDENCE TABLE (2627 events) ===


,repr_time,n_stations,stations,drop_mean,drop_max,A_complex_mean,A_complex_max
0,2018-01-01 15:48:00,1,INVK,4.115380,4.115380,5.525709,5.525709
1,2018-01-02 19:30:00,2,"JUNG1,NEWK",4.457538,4.925805,6.161780,8.011326
2,2018-01-03 20:24:00,1,LMKS,2.614254,2.614254,4.737728,4.737728
3,2018-01-04 21:30:00,2,"AATB,JUNG1",2.149605,2.256320,3.836984,3.943058
4,2018-01-05 17:49:00,2,"LMKS,NEWK",4.978914,6.494157,5.273755,5.966886



=== LOOSE SIERRA — COINCIDENCE TABLE (2982 events) ===


,repr_time,n_stations,stations,drop_mean,drop_max,A_complex_mean,A_complex_max
0,2018-01-01 19:32:00,2,"APTY,INVK",3.775539,4.133120,4.517049,5.525709
1,2018-01-02 19:30:00,5,"AATB,DOMC,JUNG1,NEWK,OULU",4.543193,6.865848,4.641459,8.011326
2,2018-01-03 20:24:00,1,LMKS,2.614254,2.614254,4.737728,4.737728
3,2018-01-04 19:28:00,5,"AATB,DOMC,INVK,JUNG1,NEWK",4.482358,11.818695,3.653533,4.111074
4,2018-01-05 22:49:00,5,"AATB,INVK,LMKS,MXCO,NEWK",4.823260,6.773046,4.584648,5.966886


# **8. Summary per year and level (events + coincidences)**

In [16]:
for level_name, ev in events_by_level.items():
    print(f"\n===== {level_name.upper()} SIERRA =====")

    if ev.empty:
        print("No events.\n")
        continue

    ev = ev.copy()
    ev["year"] = ev["onset_time"].dt.year
    events_per_year = ev.groupby("year").size()

    coinc = coincidence_by_level.get(level_name, pd.DataFrame()).copy()
    if not coinc.empty:
        coinc["year"] = coinc["repr_time"].dt.year

        coinc_all = coinc.groupby("year").agg(
            n_coinc_all=("repr_time", "size"),
            n_stations_mean_all=("n_stations", "mean"),
            n_stations_max_all=("n_stations", "max"),
        )

        coinc_ge2 = coinc[coinc["n_stations"] >= 2].groupby("year").agg(
            n_coinc_ge2=("repr_time", "size"),
            n_stations_mean_ge2=("n_stations", "mean"),
            n_stations_max_ge2=("n_stations", "max"),
        )
    else:
        coinc_all = pd.DataFrame()
        coinc_ge2 = pd.DataFrame()

    years = sorted(
        events_per_year.index
        .union(coinc_all.index)
        .union(coinc_ge2.index)
    )

    for y in years:
        n_ev = int(events_per_year.get(y, 0))

        row_all = coinc_all.loc[y] if y in coinc_all.index else None
        row_ge2 = coinc_ge2.loc[y] if y in coinc_ge2.index else None

        n_all  = int(row_all["n_coinc_all"]) if row_all is not None else 0
        m_all  = row_all["n_stations_mean_all"] if row_all is not None else float("nan")
        M_all  = int(row_all["n_stations_max_all"]) if row_all is not None else 0

        n_ge2  = int(row_ge2["n_coinc_ge2"]) if row_ge2 is not None else 0
        m_ge2  = row_ge2["n_stations_mean_ge2"] if row_ge2 is not None else float("nan")
        M_ge2  = int(row_ge2["n_stations_max_ge2"]) if row_ge2 is not None else 0

        print(
            f"{y}: events={n_ev:4d} | "
            f"coinc(all)={n_all:3d}, <stations>={m_all:4.2f}, max={M_all} | "
            f"coinc(≥2st)={n_ge2:3d}, <stations>={m_ge2:4.2f}, max={M_ge2}"
        )


===== STRICT SIERRA =====
2018: events= 118 | coinc(all)= 89, <stations>=1.11, max=2 | coinc(≥2st)= 10, <stations>=2.00, max=2
2019: events= 119 | coinc(all)= 90, <stations>=1.17, max=2 | coinc(≥2st)= 15, <stations>=2.00, max=2
2020: events= 105 | coinc(all)= 85, <stations>=1.04, max=2 | coinc(≥2st)=  3, <stations>=2.00, max=2
2021: events= 294 | coinc(all)=139, <stations>=1.19, max=3 | coinc(≥2st)= 24, <stations>=2.12, max=3
2022: events= 146 | coinc(all)=107, <stations>=1.21, max=3 | coinc(≥2st)= 20, <stations>=2.10, max=3
2023: events= 191 | coinc(all)=135, <stations>=1.20, max=3 | coinc(≥2st)= 22, <stations>=2.23, max=3
2024: events= 240 | coinc(all)=156, <stations>=1.37, max=3 | coinc(≥2st)= 50, <stations>=2.16, max=3
2025: events= 193 | coinc(all)=118, <stations>=1.35, max=3 | coinc(≥2st)= 31, <stations>=2.32, max=3

===== MEDIUM SIERRA =====
2018: events= 819 | coinc(all)=328, <stations>=2.01, max=6 | coinc(≥2st)=192, <stations>=2.72, max=6
2019: events= 752 | coinc(all)=308, <

# **9. Load IZMIRAN 2019 catalog and match to Sierra coincidences**

In [29]:
# Adjust path / delimiter to your file
izm = pd.read_csv("izmiran2019.txt", delimiter="\t")
#### Source: http://spaceweather.izmiran.ru/eng/fds2019.html
izm["t_izmiran"] = pd.to_datetime(izm["Date of event"],
                                  format="%Y.%m.%d %H:%M:%S")
izm["t_izmiran"] = izm["t_izmiran"].dt.tz_localize(None)

izm.head()

,Date of event,MagnM,Axym,Azrange,TminM,DminM,OType,Bmax,Bzmin,Vmax,Dstmin,t_izmiran
0,2019.01.04 12:00:00,1.4,1.01,0.80,62,-0.19,9,11.4,-7.1,554.0,-23.0,2019-01-04 12:00:00
1,2019.01.10 14:00:00,0.6,0.82,0.65,14,-0.23,9,7.3,-3.0,509.0,-9.0,2019-01-10 14:00:00
2,2019.01.13 01:00:00,0.4,0.84,0.50,10,-0.15,9,3.4,-2.0,366.0,-4.0,2019-01-13 01:00:00
3,2019.01.13 23:00:00,0.5,1.06,0.70,2,-0.27,9,10.2,-3.8,403.0,-2.0,2019-01-13 23:00:00
4,2019.01.15 11:00:00,0.4,1.06,0.57,15,-0.19,9,7.7,-2.6,417.0,-7.0,2019-01-15 11:00:00


In [30]:
def match_izmiran_to_sierra(izm_df,
                            coincidence_by_level,
                            tolerance_hours=24):
    """
    For each Sierra level, match each IZMIRAN event (2019)
    to the closest Sierra coincidence event within ±tolerance_hours.
    """
    tol = pd.Timedelta(hours=tolerance_hours)
    matches_by_level = {}
    summary_rows = []

    for level_name, coinc in coincidence_by_level.items():
        coinc_2019 = coinc[coinc["repr_time"].dt.year == 2019].copy()
        if coinc_2019.empty:
            matches_by_level[level_name] = pd.DataFrame()
            summary_rows.append({
                "level": level_name,
                "n_izmiran": len(izm_df),
                "n_matched": 0,
                "match_fraction": 0.0,
                "dt_median_hours": np.nan,
                "dt_mean_abs_hours": np.nan,
                "stations_mean": np.nan,
                "stations_max": 0,
            })
            continue

        rows = []
        for i, r in izm_df.iterrows():
            t0 = r["t_izmiran"]
            dt = coinc_2019["repr_time"] - t0
            dt_abs = dt.abs()

            jmin = dt_abs.idxmin()
            best_dt = dt.loc[jmin]
            best_row = coinc_2019.loc[jmin]

            if dt_abs.loc[jmin] <= tol:
                matched = True
                dt_hours = best_dt.total_seconds() / 3600.0
                n_st = best_row["n_stations"]
                st_list = best_row["stations"]
                drop_mean = best_row["drop_mean"]
                amax = best_row["A_complex_max"]
            else:
                matched = False
                dt_hours = np.nan
                n_st = np.nan
                st_list = ""
                drop_mean = np.nan
                amax = np.nan

            rows.append({
                "idx_izmiran": i,
                "t_izmiran": t0,
                "MagnM": r.get("MagnM", np.nan),
                "matched": matched,
                "dt_hours": dt_hours,
                "n_stations": n_st,
                "stations": st_list,
                "drop_mean": drop_mean,
                "A_complex_max": amax,
            })

        matches = pd.DataFrame(rows)

        matched_mask = matches["matched"]
        n_matched = matched_mask.sum()
        if n_matched > 0:
            dt_median = matches.loc[matched_mask, "dt_hours"].median()
            dt_mean_abs = matches.loc[matched_mask, "dt_hours"].abs().mean()
            st_mean = matches.loc[matched_mask, "n_stations"].mean()
            st_max = matches.loc[matched_mask, "n_stations"].max()
        else:
            dt_median = np.nan
            dt_mean_abs = np.nan
            st_mean = np.nan
            st_max = 0

        summary_rows.append({
            "level": level_name,
            "n_izmiran": len(izm_df),
            "n_matched": int(n_matched),
            "match_fraction": n_matched / len(izm_df),
            "dt_median_hours": dt_median,
            "dt_mean_abs_hours": dt_mean_abs,
            "stations_mean": st_mean,
            "stations_max": st_max,
        })

        matches_by_level[level_name] = matches

    summary_by_level = pd.DataFrame(summary_rows).set_index("level")
    return matches_by_level, summary_by_level

In [31]:
matches_by_level, summary_by_level = match_izmiran_to_sierra(
    izm_df=izm,
    coincidence_by_level=coincidence_by_level,
    tolerance_hours=24
)

summary_by_level

,n_izmiran,n_matched,match_fraction,dt_median_hours,dt_mean_abs_hours,stations_mean,stations_max
level,,,,,,,
strict,124,45,0.362903,-0.516667,11.860000,1.111111,2.0
medium,124,118,0.951613,1.616667,8.309605,1.805085,6.0
loose,124,124,1.000000,0.716667,6.716935,3.233871,9.0


# **10. Sierra-M 2019 catalog (Medium + drop ≥ 1.5%) and quality classes**

In [32]:
med_coinc = coincidence_by_level["medium"].copy()
med_coinc["repr_time"] = med_coinc["repr_time"].dt.tz_localize(None)
med_coinc_2019 = med_coinc[med_coinc["repr_time"].dt.year == 2019].copy()

# Sierra-M base catalog: drop_mean >= 1.5, at least 1 station
sierra_M_2019 = med_coinc_2019[med_coinc_2019["drop_mean"] >= 1.5].copy()
len(sierra_M_2019)

308

In [33]:
def assign_quality(row):
    nst = row["n_stations"]
    drop = row["drop_mean"]
    if nst >= 3:
        return "A"   # multi-station, most global
    elif nst == 2:
        return "B"   # bi-station
    elif (nst == 1) and (drop >= 2.0):
        return "C"   # local but strong
    else:
        return "D"   # weaker / marginal

sierra_M_2019["quality"] = sierra_M_2019.apply(assign_quality, axis=1)
sierra_M_2019["quality"].value_counts()

quality
C    136
B     88
A     82
D      2
Name: count, dtype: int64

In [34]:
# Flag which Sierra-M events have at least one IZMIRAN match (±24 h)
tol = pd.Timedelta(hours=24)
sierra_M_2019["matched_izmiran"] = False

for _, r in izm.iterrows():
    t0 = r["t_izmiran"]
    mask = (sierra_M_2019["repr_time"] >= t0 - tol) & \
           (sierra_M_2019["repr_time"] <= t0 + tol)
    sierra_M_2019.loc[mask, "matched_izmiran"] = True

# Cross-tab quality vs matched / Sierra-only
ct = pd.crosstab(sierra_M_2019["quality"],
                 sierra_M_2019["matched_izmiran"])
ct.columns = ["Sierra-only", "Matched_IZMIRAN"]
ct

,Sierra-only,Matched_IZMIRAN
quality,,
A,43,39
B,36,52
C,48,88
D,0,2


In [35]:
# Also mark in IZMIRAN which ones are captured by Sierra-M and with what qualities
flags = []
quals = []

for _, r in izm.iterrows():
    t0 = r["t_izmiran"]
    mask = (sierra_M_2019["repr_time"] >= t0 - tol) & \
           (sierra_M_2019["repr_time"] <= t0 + tol)
    if mask.any():
        flags.append(True)
        quals.append(",".join(sorted(sierra_M_2019.loc[mask, "quality"].unique())))
    else:
        flags.append(False)
        quals.append("")

izm["in_sierra_M"] = flags
izm["sierra_qualities"] = quals

len_izm = len(izm)
n_cap = izm["in_sierra_M"].sum()

print("Total IZMIRAN 2019:", len_izm)
print("Captured by Sierra-M:", n_cap, f"({n_cap/len_izm:.3f})")

izm[izm["in_sierra_M"]]["sierra_qualities"].value_counts()

Total IZMIRAN 2019: 124
Captured by Sierra-M: 118 (0.952)


sierra_qualities
C        38
B,C      21
B        16
A        15
A,C      14
A,B      10
A,B,C     2
C,D       2
Name: count, dtype: int64

# **11. Save catalogs to CSV**

In [36]:
# Save per-level single-station events
for lvl, ev in events_by_level.items():
    ev.to_csv(f"Results/sierra_events_{lvl}.csv", index=False)

# Save per-level coincidence tables
for lvl, coinc in coincidence_by_level.items():
    coinc.to_csv(f"Results/sierra_coincidences_{lvl}.csv", index=False)

# Save Sierra-M 2019 catalog
sierra_M_2019.to_csv("Results/sierra_M_2019_catalog.csv", index=False)

# Save IZMIRAN 2019 with flags
izm.to_csv("Results/izmiran_2019_with_sierra_flags.csv", index=False)

# **12. Build the AFTER-M catalog for all years**

In [38]:
import numpy as np
import pandas as pd

# If not already loaded in this notebook:
# coinc_medium = pd.read_csv("sierra_coincidences_medium.csv", parse_dates=["repr_time"])
# if pd.api.types.is_datetime64tz_dtype(coinc_medium["repr_time"]):
#     coinc_medium["repr_time"] = coinc_medium["repr_time"].dt.tz_convert(None)

# 1) Add year column
coinc_M = med_coinc.copy()
coinc_M["year"] = coinc_M["repr_time"].dt.year

# 2) Apply AFTER-M amplitude selection: mean fractional decrease >= 1.5%
mask_M = coinc_M["drop_mean"] >= 1.5
AFTER_M_all = coinc_M[mask_M].copy()

# 3) Assign quality classes A/B/C/D (same logic as for 2019)
conditions = [
    AFTER_M_all["n_stations"] >= 3,                                  # A
    AFTER_M_all["n_stations"] == 2,                                  # B
    (AFTER_M_all["n_stations"] == 1) & (AFTER_M_all["drop_mean"] >= 2.0),  # C
]
choices = ["A", "B", "C"]

AFTER_M_all["quality"] = np.select(conditions, choices, default="D")

# 4) Sort by time (optional but nice)
AFTER_M_all = AFTER_M_all.sort_values("repr_time").reset_index(drop=True)

AFTER_M_all.head()


,repr_time,n_stations,stations,drop_mean,drop_max,A_complex_mean,A_complex_max,year,quality
0,2018-01-01 15:48:00,1,INVK,4.115380,4.115380,5.525709,5.525709,2018,C
1,2018-01-02 19:30:00,2,"JUNG1,NEWK",4.457538,4.925805,6.161780,8.011326,2018,B
2,2018-01-03 20:24:00,1,LMKS,2.614254,2.614254,4.737728,4.737728,2018,C
3,2018-01-04 21:30:00,2,"AATB,JUNG1",2.149605,2.256320,3.836984,3.943058,2018,B
4,2018-01-05 17:49:00,2,"LMKS,NEWK",4.978914,6.494157,5.273755,5.966886,2018,B


In [40]:
# AFTER_M_2019 = pd.read_csv("sierra_M_2019_catalog.csv", parse_dates=["repr_time"])
AFTER_M_2019=sierra_M_2019.copy()

AFTER_M_all_2019 = AFTER_M_all[AFTER_M_all["year"] == 2019].copy()

len(AFTER_M_all_2019), len(AFTER_M_2019)


(308, 308)

# **13. Save the complete AFTER-M catalog (all years)**

In [41]:
import os

os.makedirs("Results", exist_ok=True)

AFTER_M_all.to_csv("Results/AFTER_M_full_catalog.csv", index=False)
print("Saved global AFTER-M catalog to Results/AFTER_M_full_catalog.csv")


Saved global AFTER-M catalog to Results/AFTER_M_full_catalog.csv
